# **Class 10: LangChain: Building Tools and Agents for Intelligent Interactions**

- **Introduction to LangChain Tools and Agents**
  - Overview of LangChain as a framework to extend language model capabilities
  - Explanation of Tools: External utilities and APIs for performing tasks
  - Explanation of Agents: Autonomous entities that utilize tools for goal-driven tasks

- **SQLite Database Creation for E-commerce Chatbot**
  - Set up a database to manage essential components: products, clients, and orders
  - Understand the structure and relationships between these database tables

- **Utilizing Built-in SQL Tools in LangChain**
  - Introduction to the SQLDatabaseToolkit
  - Demonstrate how to create an Agent that converts natural language queries into SQL queries
  - Generate natural language responses from the executed SQL queries

- **Creating Custom Tools in LangChain**
  - Develop `CreateOrderTool` to facilitate the creation of new orders in the orders table
  - Develop `GetOrderTool` to retrieve existing order information
  - Explore how these tools can dynamically interact with the orders table

- **Developing an Agent with Custom Tools**
  - Assemble an Agent that integrates both `CreateOrderTool` and `GetOrderTool`
  - Enable the Agent to process and respond to complex tasks related to orders
  - Showcase real-world applications of this Agent in an e-commerce context

- **Conclusion**
  - Recap of using Tools and Agents to enhance language model applications
  - Encourage experimentation with custom tools to meet specific application needs 
  - Discuss potential projects and further exploration of LangChain capabilities.

In [21]:
import pickle

# Load the products catalog from a pickle file
with open("products_catalog.pkl", "rb") as handle:
    products_catalog = pickle.load(handle)

In [22]:
products_catalog["TechPro Ultrabook"]

{'name': 'TechPro Ultrabook',
 'category': 'Computers and Laptops',
 'brand': 'TechPro',
 'model_number': 'TP-UB100',
 'warranty': '1 year',
 'rating': 4.5,
 'features': ['13.3-inch display',
  '8GB RAM',
  '256GB SSD',
  'Intel Core i5 processor'],
 'description': 'A sleek and lightweight ultrabook for everyday use.',
 'price': 799.99}

Each product has a name, category, brand, model_number, warranty, rating, features, description and price.

In [ ]:
import sqlite3

# Create the database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Create the products table
c.execute('''CREATE TABLE products
             (product_id INTEGER PRIMARY KEY,
              name TEXT,
              category TEXT,
              brand TEXT,
              model_number TEXT,
              warranty INTEGER,
              rating REAL,
              features TEXT,
              description TEXT,
              price REAL)''')

# Create the customers table
c.execute('''CREATE TABLE customers
             (customer_id INTEGER PRIMARY KEY,
              first_name TEXT,
              last_name TEXT,
              email TEXT,
              phone TEXT)''')


# Create the orders table
# Each order is associated with a customer and a product
# Create the orders table
c.execute('''CREATE TABLE orders
             (order_id INTEGER PRIMARY KEY,
              customer_id INTEGER,
              product_id INTEGER,
              quantity INTEGER,
              total_amount REAL,
              order_date TEXT,
              FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
              FOREIGN KEY (product_id) REFERENCES products(product_id))''')

conn.commit()
conn.close()

Let's populate the products table with our products catalog.

In [ ]:
import json

# Connect to the database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

for product in products_catalog.values():
    c.execute("INSERT INTO products (name, category, brand, model_number, warranty, rating, features, description, price) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
              (product["name"], product["category"], product["brand"], product["model_number"], product["warranty"], product["rating"], json.dumps(product["features"]), product["description"], product["price"]))

conn.commit()
conn.close()

In [26]:
# Connect to the database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Insert sample data
c.execute("INSERT INTO customers (first_name, last_name, email, phone) VALUES (?, ?, ?, ?)", ("John","Doe", "john.doe@example.com", "555-1234"))
c.execute("INSERT INTO customers (first_name, last_name, email, phone) VALUES (?, ?, ?, ?)", ("Jane", "Smith", "jane.smith@example.com", "555-5678"))

conn.commit()
conn.close()

In [27]:
# Connect to the database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Insert sample data
c.execute("INSERT INTO orders (customer_id, product_id, quantity, total_amount, order_date) VALUES (?, ?, ?, ?, ?)", (1, 1, 1, 799.99, "2023-09-01"))
c.execute("INSERT INTO orders (customer_id, product_id, quantity, total_amount, order_date) VALUES (?, ?, ?, ?, ?)", (2, 2, 1, 1199.99, "2023-09-15"))

conn.commit()
conn.close()

**Chains** serve as the backbone of workflow automation, organizing a sequence of operations into a coherent process that transforms input data step-by-step to achieve desired results. By linking various actions—such as parsing user input, performing calculations, and generating responses—Chains ensure a structured and efficient execution path, allowing developers to design comprehensive processes that can handle sophisticated tasks effortlessly. **Benefit from memory** by storing intermediate states or results as they process input data through their sequence of operations.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [9]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Define the prompt template for translation
prompt = ChatPromptTemplate.from_messages(
    [   
        SystemMessagePromptTemplate.from_template("Translate the following review to English."),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

In [10]:
from langchain.chat_models import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')

In [11]:
from langchain_core.output_parsers.string import StrOutputParser

# Define the output parser
output_parser = StrOutputParser()

In [12]:
chain = prompt | llm | output_parser

In [13]:
chain.invoke({"text": "Bu otelde kalmak çok güzeldi. Personel çok yardımseverdi ve odalar çok temizdi."})

'Staying at this hotel was very nice. The staff was very helpful and the rooms were very clean.'

LangChain provides `SQLDatabase` SQLAlchemy wrapper around a database.

In [28]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI

# Path to your SQLite .db file
db_path = 'ecommerce.db'

# Create a SQLDatabase object
db = SQLDatabase.from_uri(f'sqlite:///{db_path}')

llm = ChatOpenAI(model="gpt-4o-mini")

We can run SQL queries directly on the SQLDatabase instance.

In [ ]:
db.run("SELECT * FROM products")

We can the check the sql dialect used by the database and the names of the tables in the database.

In [29]:
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['customers', 'orders', 'products']


`create_sql_query_chain` create a chain that generates SQL queries.

In [30]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "What are the products?"})

In [31]:
response

'SQLQuery: SELECT "product_id", "name", "category", "brand", "price" FROM products LIMIT 5;'

**Tools** provide the necessary functional extensions that allow language models to interact with external systems, such as databases or APIs. By integrating these specialized utilities, Tools enable language models to perform actions beyond text manipulation, such as querying databases, retrieving web data, or executing complex computations. This expansion of capabilities allows for the creation of applications that can operate effectively in diverse scenarios, making language models more versatile and applicable to a wide range of practical applications. **Typically operate without intrinsic memory**, focusing instead on providing specific, stateless services or computations

LangChain has a built-in `SQLDatabaseToolkit` that provides tools for interacting with SQL databases.

In [32]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [33]:
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001CAFF489E50>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001CAFF489E50>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001CAFF489E50>),
 QuerySQLCheckerTool(description='Use this tool to 

In [34]:
from langchain import hub

# Download the SQL prompt template from the hub
sql_prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

print(sql_prompt_template.input_variables)

['dialect', 'top_k']


In [35]:
sql_prompt_template.pretty_print()

================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th


**Agents** act as intelligent orchestrators within the LangChain framework, autonomously managing tasks by employing the appropriate Tools and executing the necessary Chains. They interpret user inputs or environmental cues, decide on the optimal course of action, and dynamically integrate Tools and Chains to fulfill tasks while adapting to changes in context and requirements. This makes Agents the key facilitators of decision-making and problem-solving, driving the interaction between users and the underlying technology to produce intelligent, context-aware, and responsive systems. **Use memory**, as they can maintain context over extended interactions, enabling more personalized and coherent responses.

In [36]:
from langchain import hub

# Doqnload a AgentPrompt from the hub
agent_prompt = hub.pull("hwchase17/openai-functions-agent")
print(agent_prompt.input_variables)

['agent_scratchpad', 'input']


In [37]:
agent_prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


Let's replace the system message with the SQL Agent system message.

In [39]:
from langchain.prompts import SystemMessagePromptTemplate

system_message = sql_prompt_template.format(dialect="SQLite", top_k=5)

system_prompt = SystemMessagePromptTemplate.from_template(system_message)

agent_prompt.messages[0] = system_prompt

In [40]:
agent_prompt.pretty_print()

================================ System Message ================================

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the

Now let's create our first Agent.

In [41]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, toolkit.get_tools(), agent_prompt)

Finally, we combine the agent (the brains) with the tools inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=toolkit.get_tools())

In [43]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [ ]:
response = agent_executor.invoke({"input": "What are the available products?"})

In [48]:
print(response["output"])

Here are some of the products available:

1. **MobiTech Wireless Charger**
   - Category: Smartphones and Accessories
   - Brand: MobiTech
   - Price: $29.99

2. **WaveSound Bluetooth Speaker**
   - Category: Audio Equipment
   - Brand: WaveSound
   - Price: $49.99

3. **MobiTech PowerCase**
   - Category: Smartphones and Accessories
   - Brand: MobiTech
   - Price: $59.99

4. **ProGamer Controller**
   - Category: Gaming Consoles and Accessories
   - Brand: ProGamer
   - Price: $59.99

5. **FotoSnap Instant Camera**
   - Category: Cameras and Camcorders
   - Brand: FotoSnap
   - Price: $69.99


The SQLToolKit in LangChain is primarily equipped for executing data retrieval via SELECT queries from databases. However, it does not support Data Manipulation Language (DML) operations like INSERT, UPDATE, DELETE, or schema modifications such as DROP. To handle these DML tasks, developers need to create custom Tools by subclassing from the `BaseTool` class. 

In our session, we will demonstrate this process by developing custom Tools specifically designed for managing orders within our application. We will create a `CreateOrderTool` to facilitate the addition of new orders and a `GetOrderTool` to retrieve existing order details. These custom tools will enable us to interact dynamically with our database, enhancing the functionality of our e-commerce chatbot beyond simple data queries.

In [49]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

products = query_as_list(db, "SELECT Name FROM products")
descriptions = query_as_list(db, "SELECT description FROM products")

In [ ]:
# Import necessary modules and classes
from pydantic import BaseModel, Field
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)


class PromptTemplate(BaseModel):
    """Defines templates for system and human messages used in a conversation."""

    system_template: str = Field(
        description="Template for the system message in the conversation"
    )
    human_template: str = Field(
        description="Template for the human message in the conversation"
    )


def generate_prompt_templates(
    prompt_template: PromptTemplate
) -> ChatPromptTemplate:
    """Generate a chat prompt template based on given templates and memory setting.

    Args:
        prompt_template: An instance of PromptTemplate containing system and human templates.

    Returns:
        A configured ChatPromptTemplate with specified message structure.
    """

    # Create prompt template without chat history
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(
                prompt_template.system_template
            ),
            HumanMessagePromptTemplate.from_template(
                prompt_template.human_template
            ),
        ]
    )

    return prompt

For our custom order management Tools, `CreateOrderReasoningChain` and `GetOrderReasoningChain`, we will enhance their functionality by developing specialized Chains that extract pertinent information from user input.

- **CreateOrderReasoningChain**: This Chain will be designed to gather all necessary details required for creating a new order, such as product ID, quantity, customer information, and any additional preferences. By systematically parsing and structuring user inputs, the CreateOrderChain ensures that the data passed to the `CreateOrderTool` is complete and ready for execution, allowing for seamless order creation within the system.~

In [54]:
from langchain.schema.runnable.base import Runnable
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from pydantic import BaseModel
from langchain_community.utilities.sql_database import SQLDatabase
import ast
import re


class OrderInformation(BaseModel):
    product_name: str
    quantity: int


class CreateOrderReasoningChain(Runnable):
    def __init__(self, llm):
        super().__init__()

        self.llm = llm
        # Path to your SQLite .db file
        db_path = "ecommerce.db"

        self.db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
        self.products_list = self.query_as_list("SELECT Name FROM products")

        prompt_template = PromptTemplate(
            system_template=""" 
            You are a part of the e-commerce team. 
            Your task is to identify the product name and quantity from the user input.

            Here is the list of available products:
            {products_list}

            Here is the user input:
            {customer_input}

            {format_instructions}
            """,
            human_template="Customer Query: {customer_input}",
        )

        self.prompt = generate_prompt_templates(prompt_template)
        self.output_parser = PydanticOutputParser(pydantic_object=OrderInformation)
        self.format_instructions = self.output_parser.get_format_instructions()

        self.chain = self.prompt | self.llm | self.output_parser

    def query_as_list(self, query):
        res = self.db.run(query)
        res = [el for sub in ast.literal_eval(res) for el in sub if el]
        res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
        return list(set(res))

    def invoke(self, inputs):
        return self.chain.invoke(
            {
                "customer_input": inputs["customer_input"],
                "products_list": self.products_list,
                "format_instructions": self.format_instructions,
            },
        )

In [55]:
creator_order_chain = CreateOrderReasoningChain(llm)

In [56]:
creator_order_chain.invoke({"customer_input": "I want to order 2 TechPro Ultrabooks."})

OrderInformation(product_name='TechPro Ultrabook', quantity=2)

- **GetOrderReasoningChain**: Similarly, this Chain will focus on extracting key information needed to retrieve specific order details, like order ID or customer name. The GetOrderChain will process user requests to accurately identify and format the input data, enabling the `GetOrderTool` to efficiently fetch and return the requested order information from the database.

In [59]:
class OrderId(BaseModel):
    order_id: int


class GetOrderReasoningChain(Runnable):
    def __init__(self, llm):
        super().__init__()

        self.llm = llm
        prompt_template = PromptTemplate(
            system_template=""" 
            You are a part of the e-commerce team. 
            Your task is to identify the order_id from the user input.

            Here is the user input:
            {customer_input}

            {format_instructions}
            """,
            human_template="Customer Query: {customer_input}",
        )

        self.prompt = generate_prompt_templates(prompt_template)
        self.output_parser = PydanticOutputParser(pydantic_object=OrderId)
        self.format_instructions = self.output_parser.get_format_instructions()

        self.chain = self.prompt | self.llm | self.output_parser

    def invoke(self, inputs):
        return self.chain.invoke(
            {
                "customer_input": inputs["customer_input"],
                "format_instructions": self.format_instructions,
            },
        )


In [60]:
get_order_chain = GetOrderReasoningChain(llm)

In [62]:
get_order_chain.invoke({"customer_input": "I want to know the status of order id 1."})

OrderId(order_id=1)

Now let's finally create our custom tools.
First we will create the `CreateOrderTool` and then the `GetOrderTool`.

In [ ]:
from typing import Type, Any
from pydantic import BaseModel
from langchain.tools import BaseTool
import sqlite3 
from langchain_openai import ChatOpenAI


class CreateOrderInput(BaseModel):
    customer_id: int
    customer_input: str


class CreateOrderTool(BaseTool):
    name: str = "CreateOrderTool" # This should only contain letters
    description: str = "Create a new order in the e-commerce database"
    args_schema: Type[BaseModel] = CreateOrderInput
    return_direct: bool = True

    def _run(
        self,
        customer_id: int,
        customer_input: str,
    ) -> str:
        llm = ChatOpenAI(model="gpt-4o-mini")

        order_info = CreateOrderReasoningChain(llm).invoke({"customer_input": customer_input})

        connection = sqlite3.connect('ecommerce.db')
        cursor = connection.cursor()

        try:
            cursor.execute("SELECT product_id, price FROM products WHERE name = ?", (order_info.product_name,))
            product_id, price = cursor.fetchone()

            total_amount = price * order_info.quantity
            order_date = '2023-05-01'
            quantity = order_info.quantity

            
            cursor.execute(
                "INSERT INTO orders (customer_id, product_id, quantity, total_amount, order_date) VALUES (?, ?, ?, ?, ?)",
                (customer_id, product_id, quantity, total_amount, order_date),
            )
            connection.commit()

        except sqlite3.OperationalError as e:
            print(f"Error: {e}")
        finally:
            cursor.close()
            connection.close()

        
        return f"Order created with ID: {cursor.lastrowid}"

In [64]:
create_order_tool = CreateOrderTool()

In [66]:
new_order_dict = {
    "customer_id": 1,
    "customer_input": "I would like to order 2 TechPro Ultrabooks."
}
create_order_tool.invoke(new_order_dict)

'Order created with ID: 3'

In [70]:
from typing import Type, Any
from pydantic import BaseModel
from langchain.tools import BaseTool
import sqlite3


class GetOrderInput(BaseModel):
    customer_id: int
    customer_input: str


class GetOrderTool(BaseTool):
    name: str = "GetOrderTool"
    description: str = "Retrieve details of an existing order based on the order ID"
    args_schema: Type[BaseModel] = GetOrderInput
    return_direct: bool = True

    def _run(
        self,
        customer_id: int,
        customer_input: str,
    ) -> str:
        llm = ChatOpenAI(model="gpt-4o-mini")

        order_info = GetOrderReasoningChain(llm).invoke({"customer_input": customer_input})
        order_id = order_info.order_id

        connection = sqlite3.connect('ecommerce.db')
        cursor = connection.cursor()

        try:
            # Get the column names of the orders table
            cursor.execute("PRAGMA table_info(orders)")
            columns = cursor.fetchall()

            cursor.execute("SELECT * FROM orders WHERE order_id = ?", (order_id,))
            order_ = cursor.fetchone()

            # Map the order to the column names
            order = dict(zip([column[1] for column in columns], order_))

        except sqlite3.OperationalError as e:
            print(f"Error: {e}")
        finally:
            cursor.close()
            connection.close()

        if customer_id != order["customer_id"]:
            return "You are not authorized to view this order."
        else:
            return order


In [71]:
check_order_tool = GetOrderTool()

In [72]:
check_order_tool.invoke({"customer_id": 1, "customer_input": "I want to know the status of order id 3."})

{'order_id': 3,
 'customer_id': 1,
 'product_id': 1,
 'quantity': 2,
 'total_amount': 1599.98,
 'order_date': '2023-05-01'}

Now let's plug these custom tools into our Agent.


In [124]:
system_message = """
You are now connected to the e-commerce database. You can use the following tools to interact with the database:

1. Create Order: Create a new order in the database
2. Get Order: Retrieve details of an existing order based on the order ID

The user_id is 
{customer_id}

If none of the above tools are needed, you can answer the customer in a polite manner.
"""

human_template = """
Customer Query: {customer_input}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_message),
        HumanMessagePromptTemplate.from_template(human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [125]:
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent

llm = ChatOpenAI(model="gpt-4o-mini")
tools = [create_order_tool, check_order_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [126]:
agent_executor.invoke({"customer_id": 1, "customer_input": "I would like to order a TechPro Ultrabook."})

{'customer_id': 1,
 'customer_input': 'I would like to order a TechPro Ultrabook.',
 'output': 'Order created with ID: 8'}

In [127]:
agent_executor.invoke({"customer_id": 1, "customer_input": "I would like to know the status of order 4."})

{'customer_id': 1,
 'customer_input': 'I would like to know the status of order 4.',
 'output': {'order_id': 4,
  'customer_id': 1,
  'product_id': 1,
  'quantity': 1,
  'total_amount': 799.99,
  'order_date': '2023-05-01'}}

Adding Memory to the Agent to store the order details.

In [128]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In-memory implementation of chat message history.

    Stores a list of messages within the session.
    """

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the in-memory store."""
        self.messages.extend(messages)

    def clear(self) -> None:
        """Clear all messages from the in-memory store."""
        self.messages = []

In [129]:
store = {}

In [130]:
def get_session_history(
    user_id: str, conversation_id: str
) -> BaseChatMessageHistory:
    """Retrieve or create session history for a specific user, conversation.

    Args:
        user_id: Identifier for the user.
        conversation_id: Identifier for the conversation.

    Returns:
        An instance of BaseChatMessageHistory for managing the chat history.
    """
    if (user_id, conversation_id) not in store:
        # Initialize new in-memory history if not already stored
        store[(user_id, conversation_id)] = InMemoryHistory()

    return store[(user_id, conversation_id)]

In [131]:
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory

memory_config = {
    "configurable": {
        "user_id": 1,
        "conversation_id": 1,
    }
}

history_factory_config = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for the user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for the conversation.",
        default="",
        is_shared=True,
    ),
]

In [132]:
system_message = """
You are now connected to the e-commerce database. You can use the following tools to interact with the database:

1. Create Order: Create a new order in the database
2. Get Order: Retrieve details of an existing order based on the order ID

The user_id is 
{customer_id}

If none of the above tools are needed, you can answer the customer in a polite manner.
"""

human_template = """
Customer Query: {customer_input}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template(human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [133]:
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent

llm = ChatOpenAI(model="gpt-4o-mini")
tools = [create_order_tool, check_order_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [134]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="customer_input",
    history_messages_key="chat_history",
    history_factory_config=history_factory_config,
)

In [135]:
user_input = "Hello my name is Tiago."

In [136]:
agent_with_chat_history.invoke(
    {
        "customer_id": 1,
        "customer_input": user_input,
        "conversation_id": 1,
    },
    config=memory_config,
)

{'customer_id': 1,
 'customer_input': 'Hello my name is Tiago.',
 'conversation_id': 1,
 'chat_history': [],
 'output': 'Hello Tiago! How can I assist you today?'}

In [137]:
user_input = "What is my name?"

In [138]:
agent_with_chat_history.invoke(
    {
        "customer_id": 1,
        "customer_input": user_input,
        "conversation_id": 1,
    },
    config=memory_config,
)

{'customer_id': 1,
 'customer_input': 'What is my name?',
 'conversation_id': 1,
 'chat_history': [HumanMessage(content='Hello my name is Tiago.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Tiago! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Tiago. How can I help you further?'}